In [1]:
import os, sys

cwd = os.getcwd()
module_path = os.path.abspath(os.path.join('..'))

sys.path.insert(0, os.path.join(cwd, "../"))
sys.path.insert(0, os.path.join(cwd, "../student_life/"))
sys.path.insert(0, os.path.join(cwd, "../student_life/src/"))
sys.path.insert(0, os.path.join(cwd, "../student_life/src/bin/"))

import copy
from scipy import stats
import pickle
import importlib
import src.utils.student_utils as student_utils
from collections import defaultdict
import geopy.distance
from sklearn.metrics.pairwise import euclidean_distances

import src.definitions as definitions
import pandas as pd
import numpy as np
from IPython.display import display
from src.data_manager import student_life_var_binned_data_manager
importlib.reload(student_life_var_binned_data_manager)
importlib.reload(student_utils)

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline  

print(module_path)

Student ID couldn't be converted to Integer!
Student ID couldn't be converted to Integer!
/Users/nsimsiri/Documents/code/ml/MultiRes/student_life


In [2]:
import geopy.distance as euclidean_distances
COORD_DIST = lambda p1, p2: euclidean_distances.distance(p1, p2).km
from scipy.spatial import ConvexHull
from area import area
from editdistance import eval as edit_distance
from sklearn.cluster import MeanShift, estimate_bandwidth
import scipy
from datetime import datetime, timedelta
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [3]:
keyword = 'gps'
def load_features(student_id=2):
    path = "{}/data/student_life_minimal_processed_data/student_{}".format(module_path, student_id)
    assert(os.path.exists(path))
    feats = {}
    for _file in os.listdir(path):
        if keyword not in _file and 'stress' not in _file: 
            continue
        feat_path = path + "/" + _file
        df = pd.read_csv(feat_path)
        print('student {} loaded feat: {}'.format(student_id, _file))      
        feat_name = _file.replace('.csv', '')
        feats[feat_name] = df
    return feats
        
    
# feats = load_features(student_id=46)
# print(len(feats))

Location Features (8 total):
 - total distance covered in a day
 - maximum 2-point displacement in a day
 - distance standard deviatio
 - number of different areas visited by tiles approximation
 - total spatial coverage by convex hull,
 - difference in sequence of tiles covered compared to previous day
 - difference in sequence of clusters visited compared to previous day
 - distance entropy

4 Temporal one-hot features 
- weekends, start of term, mid-term, end of term

https://www.ucl.ac.uk/~ucfamus/papers/digitalbiomarkers17.pdf

In [4]:
def index_time_to_df(df):
    df.loc[:, 'time'] = pd.to_datetime(df.loc[:,'time'])
    df = df.set_index('time')
    return df
# location = feats['gps_details']
# print(len(location))
# display(location[:5])
# location = index_time_to_df(location)

### (1, 2) Total Distance Covered, Max Displacement 

In [5]:
def to_distance(_location):
    location = _location.copy()
    location['distance'] = [0 for x in range(len(location))]

    for i in range(1, len(location)):
        row_cur = location.iloc[i]
        row_prev = location.iloc[i-1]
        p1 = (row_cur['latitude'], row_cur['longitude'])
        p2 = (row_prev['latitude'], row_prev['longitude'])
        dist = euclidean_distances.distance(p1, p2).km
        location.iloc[i, location.columns.get_loc('distance')]= dist
    return location

def get_feat_distance_covered(distance):
    # display(conversation)
    bin_freq = 1440# 24 * 60
    feats = distance['distance']
    sample_rule = '{}T'.format(bin_freq)
    aggregate = pd.DataFrame(feats.resample(sample_rule).sum())
    displacement = pd.DataFrame(feats.resample(sample_rule).max())
#     displacement = displacement.fillna(0.0)
    displacement = displacement.ffill()
    displacement = displacement.rename(columns={'distance': "displacement"})
    return aggregate, displacement

# distance = to_distance(location)
# total_distance, max_displacement = get_feat_distance_covered(distance)
# display(max_displacement[:3])
# display(total_distance[:3])

### (3) Standard Deviation

In [6]:
def get_std_distance(distance):
    # display(conversation)
    bin_freq = 1440# 24 * 60
    feats = distance['distance']
    sample_rule = '{}T'.format(bin_freq)
    aggregate = feats.resample(sample_rule).std()
    aggregate = aggregate.ffill()
    aggregate = pd.DataFrame(aggregate)
    aggregate = aggregate.rename(columns={'distance': "distance_std"})
    return aggregate

# std_distances = get_std_distance(distance)
# display(std_distances[:3])

### (4) Spatial Tile - unique tile visited

In [7]:

def build_spatial_map(_location, bound=50):
    location = _location.copy()
    lats  = location['latitude']
    longs = location['longitude'] 
    min_lat, max_lat = lats.min(), lats.max()
    min_long, max_long = longs.min(), longs.max()
#     print("min_lat", min_lat, "max_lat", max_lat, "min_long", min_long, "max_long", max_long)
    TL = [min_lat, max_long]
    TR = [max_lat, max_long]
    BL = [min_lat, min_long]
    BR = [max_lat, min_long]
#     print(COORD_DIST(TL, TR), COORD_DIST(BL, BR), abs(COORD_DIST(TL, TR)-COORD_DIST(BL, BR)))
#     print(COORD_DIST(TL, BL), COORD_DIST(TR, BR), abs(COORD_DIST(TL, BL)-COORD_DIST(TR, BR)))
    location['area'] = ["" for x in range(len(location))]
    for i in range(len(location)):
        row = location.iloc[i]
        lat, long = row['latitude'], row['longitude']
        x_dist = COORD_DIST([lat, long], [min_lat, long])
        y_dist = COORD_DIST([lat, long], [lat, min_long])
        hash_x, hash_y = int(x_dist/bound), int(y_dist/bound)
        hash_xy = "{}-{}".format(hash_x, hash_y)
        location.iloc[i, location.columns.get_loc('area')] = hash_xy
    return location['area']

def bin_areas_by_day(areas):
    def bin_day(subset):
        tiles = subset.values
        return tiles
    sample_rule = '{}T'.format(1440)
    aggregate = areas.resample(sample_rule).apply(bin_day)
    aggregate = pd.DataFrame(aggregate)
    return aggregate

# areas = build_spatial_map(location)
# area_days = bin_areas_by_day(areas)
# display(area_days[:3])

In [8]:
def unique_tile_feats(area_days):
    out = area_days.copy()
    N = len(area_days)
    out['unique_tile_count'] = [0 for x in range(N)]
    for i in range(N):
        tile_set = set(out.iloc[i, out.columns.get_loc('area')])
        out.iloc[i, out.columns.get_loc('unique_tile_count')] = len(tile_set)
    return out
        

# unique_tiles = unique_tile_feats(area_days)
# display(unique_tiles[:5])
# print(unique_tiles.iloc[4].values)

### (5) Convex Hull

In [9]:
def convex_hull_feats(locations, bin_freq=1440):
    def bin_convex(subset):
        coords = subset[['latitude','longitude']].values
        if(len(coords)<=2):
            return 0
        hull = ConvexHull(coords)
        points = [list(hull.points[ix]) for ix in hull.vertices]
        obj = {'type':'Polygon','coordinates':[points]}
        area_result = area(obj)/100
        return area_result
    out = locations.copy()
    sample_rule = "{}T".format(bin_freq)
    out = out.resample(sample_rule).apply(bin_convex)
    out = pd.DataFrame(out)
    out = out.rename(columns={0: "convex_hull_area"})
    return out
    

# convex_areas = convex_hull_feats(location)
# display(convex_areas[:6])


### (6) difference in sequence of tiles covered compared to previous day

In [10]:
def prev_edit_dist_feats(area_days):
    N = len(area_days)
    prev_tile_changes = area_days.copy()
    prev_tile_changes['tile_change'] = [0 for i in range(N)]
    for i in range(1, N):
        cur_tiles  = prev_tile_changes.iloc[i, prev_tile_changes.columns.get_loc('area')]
        prev_tiles = prev_tile_changes.iloc[i-1, prev_tile_changes.columns.get_loc('area')]
        change = edit_distance(cur_tiles, prev_tiles)
        prev_tile_changes.iloc[i, prev_tile_changes.columns.get_loc('tile_change')] = change
    return pd.DataFrame(prev_tile_changes['tile_change'])

# tile_changes = prev_edit_dist_feats(area_days)
# display(tile_changes[:3])

### (7) difference in sequence of clusters visited compared to previous day

In [11]:
def location_cluster_feats(locations, bin_freq=1440):
    def bin_location_cluster(subset):
        X = np.array(subset['cluster'].values.tolist())
        if (len(X)<=0): 
            return 0
        ms = MeanShift()
        ms.fit(X)
        c = len(ms.cluster_centers_)
        return c
    
    out = locations.copy()
    out['cluster'] = locations[['latitude','longitude']].values.tolist()
    bin_rule = "{}T".format(bin_freq)
    out = pd.DataFrame(out.resample(bin_rule).apply(bin_location_cluster))
    out = out.rename(columns={0: "cluster"})
    return out
    
# location_cluster = location_cluster_feats(location)
# display(location_cluster[:3])


### (8) Shannon Entropy every 10 minutes per day

In [12]:
def entropy_feats(distance, bin_freq=1440):
    def bin_shannon(subset):
        x = subset['distance'].values
        entropy = scipy.stats.entropy(x)
        return entropy
    
    out = pd.DataFrame(distance['distance'].copy())
    out = out.resample("10T").sum()
    out = out.resample("{}T".format(bin_freq)).apply(bin_shannon)
    out = out.fillna(0.0)
    out = pd.DataFrame(out)
    out = out.rename(columns={0: "entropy"})
    return out
    
# display(distance)
# entropies = entropy_feats(distance)
# display(entropies[:6])

### (9-12) Temporal Features

In [13]:
TO_YEAR = lambda x: x.astype('datetime64[Y]').astype(int) + 1970
TO_MONTH = lambda x: x.astype('datetime64[M]').astype(int)  % 12 + 1
TO_DAY = lambda x: x.astype('datetime64[D]').astype(int)
def to_year_month_date(x):
    year, month, day = str(x).split('T')[0].split("-")
    year, month, day = int(year), int(month), int(day)
    return year, month, day

def is_weekend(dt64):
    y,m,d = to_year_month_date(dt64)
    x = datetime(y,m,d,0,9)
    return x.isoweekday() in [6,7]

def temporal_feats(distances):
    N = len(distances)
    out = distances.copy()
    mid_start_raw = '2013-04-15T00:00:00.000000000'#Monday, April 15, 2013
    mid_end_raw = '2013-05-06T00:00:00.000000000'# Monday, May 6, 2013
    mid_start = np.datetime64(mid_start_raw)
    mid_end   = np.datetime64(mid_end_raw)
    out['start_term'] = [0 for i in range(N)]
    out['end_term'] = [0 for i in range(N)]
    out['mid_term'] = [0 for i in range(N)]
    out['weekends'] = [0 for i in range(N)]
    for idx in out.index.values:
        weekend_col = out.columns.get_loc('weekends')
        out.at[idx,'weekends'] = int(is_weekend(idx))
        x_term_col = ""
        if (idx <= mid_start):
            x_term_col = 'start_term'
        elif (idx >= mid_end):
            x_term_col = 'end_term'
        else:
            x_term_col = 'mid_term'
        out.at[idx, x_term_col] = 1
    return out
        
        
# temporal_data = temporal_feats(total_distance)
# display(temporal_data[:5])

### stress 

In [14]:
def get_stress_labels(feats, bin_freq=1440):
    def curry_apply(stress_median):
        def bin_stress_to_classes(subset):
            stress_mean = np.mean(subset['stress_level'])
            if stress_mean < stress_median: 
                return 0
            elif stress_mean > stress_median:
                return 2
            else:
                return 1
        return bin_stress_to_classes
    
    stress = feats['stress_details']

    stress_median = stress['stress_level'].median()
    stress = index_time_to_df(stress)
    stress_labels = stress.resample("{}T".format(bin_freq)).apply(curry_apply(stress_median))
    stress_labels = stress_labels.ffill()
    stress_labels = pd.DataFrame(stress_labels)
    stress_labels = stress_labels.rename(columns={0: "stress"})
    return stress_labels
    
# stress_labels = get_stress_labels(feats)
# display(stress_labels[:3])

In [15]:

import time
def build_features(feats, student_id=2):
    t0 = time.time()
    location = feats['gps_details']
    location = index_time_to_df(location)
    # 1,2,3 - basic stats
    distance = to_distance(location)
    total_distance, max_displacement = get_feat_distance_covered(distance)
    std_distances = get_std_distance(distance)
    # 4 - unique tiles
    areas = build_spatial_map(location)
    area_days = bin_areas_by_day(areas)
    unique_tiles = unique_tile_feats(area_days)
    # 5 - edit-disatance tiles
    tile_changes = prev_edit_dist_feats(area_days)
    # 6 - convex
    convex_areas = convex_hull_feats(location)
    # 7 - cluster
    location_cluster = location_cluster_feats(location)
    # 8 - entropy
    entropies = entropy_feats(distance)
    # 9,10,11,12 - temporal
    temporal_data = temporal_feats(total_distance)

    print('student {} features built: {} sec'.format(student_id, time.time()-t0))
    feature_set = pd.concat([total_distance, max_displacement, std_distances, unique_tiles, tile_changes, 
                             convex_areas, location_cluster, entropies, temporal_data], axis=1, sort=False)
    
    features_list = ['distance', 'displacement', 'distance_std', 'unique_tile_count', 'convex_hull_area',
                    'tile_change','cluster', 'entropy', 'start_term', 'end_term', 'mid_term', 'weekends'] 
    feature_set = pd.DataFrame(feature_set[features_list])
    feature_set = feature_set.loc[:,~feature_set.columns.duplicated()]
    return feature_set
    
# build_features(student_id=43)

In [16]:
# labels - stress
def build_dataset(student_id=43):
#     student_id = 43
    feats = load_features(student_id=student_id)
    stress_labels = get_stress_labels(feats)
    features = build_features(feats, student_id = student_id)
    dataset = pd.concat([features, stress_labels], axis=1, join='inner', sort=False)
    dataset['student_id'] = [student_id for i in range(len(dataset))]
    dataset = dataset.reset_index()
    return dataset, features, stress_labels

# dataset, features, stress = build_dataset(student_id=43)


In [17]:
STUDENTS_LIST = [1, 2,4 , 7,10, 22, 24, 33, 42, 46, 49, 53, 57]
datasets = []
for sid in STUDENTS_LIST:
    dataset, _, _ = build_dataset(student_id=sid)
    print("student {} data-length: {}".format(sid, len(dataset)))
    datasets.append(dataset)


student 1 loaded feat: stress_details.csv
student 1 loaded feat: gps_details.csv
student 1 features built: 9.778032779693604 sec
student 1 data-length: 39
student 2 loaded feat: stress_details.csv
student 2 loaded feat: gps_details.csv
student 2 features built: 11.582826137542725 sec
student 2 data-length: 42
student 4 loaded feat: stress_details.csv
student 4 loaded feat: gps_details.csv
student 4 features built: 12.71067476272583 sec
student 4 data-length: 44
student 7 loaded feat: stress_details.csv
student 7 loaded feat: gps_details.csv
student 7 features built: 5.496417045593262 sec
student 7 data-length: 25
student 10 loaded feat: stress_details.csv
student 10 loaded feat: gps_details.csv
student 10 features built: 14.082972764968872 sec
student 10 data-length: 56
student 22 loaded feat: stress_details.csv
student 22 loaded feat: gps_details.csv
student 22 features built: 13.566562175750732 sec
student 22 data-length: 55
student 24 loaded feat: stress_details.csv
student 24 loade

In [18]:
# display(datasets[0][:3])
# display(datasets[1][:3])
all_dataset = pd.concat(datasets, axis=0, sort=False)
all_dataset = all_dataset.sample(frac=1)
display(len(all_dataset))

636

In [19]:
data_path = module_path + "/data"
file_path = data_path + "/gatis.pkl"
print(file_path)
print(os.path.exists(data_path))
all_dataset.to_pickle(file_path)

/Users/nsimsiri/Documents/code/ml/MultiRes/student_life/data/gatis.pkl
True


In [20]:
mm = pd.read_pickle(file_path)
display(mm)

,time,distance,displacement,distance_std,unique_tile_count,convex_hull_area,tile_change,cluster,entropy,start_term,end_term,mid_term,weekends,stress,student_id
11,2013-04-07,0.882327,0.322038,0.046489,1,59.677504,2,4,2.328591,1,0,0,1,2,24
2,2013-04-01,4.475847,0.748614,0.169167,1,639.564543,1,6,2.464626,1,0,0,0,0,7
0,2013-03-27,5.367546,0.932402,0.207269,1,2107.786910,0,4,2.571405,1,0,0,0,0,4
50,2013-05-16,5.561188,0.504723,0.099349,1,709.321569,37,9,3.419411,0,1,0,0,1,57
33,2013-04-29,5.604574,0.655215,0.174368,1,1314.325793,7,6,2.827680,0,0,1,0,2,57
58,2013-05-26,31.231215,0.816242,0.270430,1,977.316718,10,6,3.530261,0,1,0,1,1,49
37,2013-05-03,4.100417,0.708169,0.150812,1,1267.939090,6,6,2.638583,0,0,1,0,1,2
22,2013-04-21,3.148268,0.558329,0.123540,1,608.984636,2,5,2.464960,0,0,1,1,1,22
25,2013-04-21,0.000000,347.153169,0.177296,0,0.000000,1,0,0.000000,0,0,1,1,1,4
26,2013-04-22,3.116225,0.658880,0.112052,1,869.071192,0,6,2.804787,0,0,1,0,1,1
